<a href="https://colab.research.google.com/github/kushalshah0/colab_tools/blob/main/idToJson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import json
from collections import OrderedDict
import time

def fetch_user_details(user_id):
    url = "#"
    params = {"u": user_id}
    response = requests.get(url, params=params)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        image_url = soup.select_one("#ContentPlaceHolder1_studentImage img")['src']
        rows = soup.select("table.table-striped.table-bordered tbody tr")
        data = OrderedDict()
        data["Image"] = image_url
        for row in rows:
            key = row.find_all('td')[0].text.strip()
            value = row.find_all('td')[1].text.strip()
            data[key] = value
        return data
    else:
        print(f"Failed to retrieve data for user ID {user_id}: {response.status_code}")
        return None

start_user_id =
end_user_id =

all_user_details = []

for user_id in range(start_user_id, end_user_id + 1):
    user_details = fetch_user_details(user_id)
    if user_details:
        all_user_details.append(user_details)
    time.sleep(1)

json_data = json.dumps(all_user_details, indent=4)

with open("user_details.json", "w") as json_file:
    json_file.write(json_data)

print("User details have been successfully fetched and saved to user_details.json")

In [ ]:
cat user_details.json